## ACDC dataset

In [1]:
import os
# import sys
import h5py
import numpy as np
import nibabel as nib
from tqdm import tqdm
from fnmatch import fnmatch

from matplotlib import pyplot as plt
from ipywidgets import interact,fixed

In [2]:
root = "/home/arun/datasets/ACDC_training/"
out_path = root+"h5/"
pattern = '*_4d.nii.gz'

volume_list = []

dirnames = list(os.walk(root))

for path, subdirs, files in os.walk(root):

    for name in files:
        
        if fnmatch(name, pattern):
            volume_path = os.path.join(path, name)
            volume_list.append(volume_path)

print ("No of volumes:{}".format(len(volume_list)))

No of volumes:100


In [3]:
# for iter,vol_path in enumerate(tqdm(volume_list)):

#     inp_vol_path = vol_path 
#     imgvol  = nib.load(inp_vol_path)
#     imgvol_data_flipped = imgvol.get_fdata()
    
#     imgvol_data = np.flip(imgvol_data_flipped,axis=1)

#     start_slice = 30
#     end_slice   = imgvol_data.shape[0] - 70

#     imgvol = imgvol_data[8:248,16:256,start_slice:end_slice]
#     #imgvol = imgvol_data[start_slice:end_slice,:,:]
#     #imgvol = np.transpose(imgvol,(1,2,0))

#     save_path = os.path.join(out_path,'{}.h5'.format(iter))
    
#     # with h5py.File(save_path,'w') as f:
#     #     f.create_dataset('inpVol',data=imgvol)


for iter,vol_path in enumerate(tqdm(volume_list)):

    inp_vol_path = vol_path
    patient_name = inp_vol_path.split("/")[-1].split(".")[0]
    
    splits = inp_vol_path.split("/")
    splits[-1] = "Info.cfg"
    cfg_path = '/'.join(splits)
    
    with open(cfg_path) as f:
        lines = f.readlines()
    
    ED_start_frame = lines[0][-2]
    
    splits = inp_vol_path.split("/")
    splits[-1] = splits[-2]+"_frame0{}_gt.nii.gz".format(ED_start_frame)
    gt_vol_path = '/'.join(splits)    
    
    imgvol  = nib.load(inp_vol_path)
    imgvol = imgvol.get_fdata()
    
    gtvol = nib.load(gt_vol_path)
    gtvol = gtvol.get_fdata()
    
    gtvol[gtvol > 0] = 1
    x_ind,y_ind,z_ind = np.where(gtvol)
    
    min_x_ind = np.min(x_ind)
    max_x_ind = np.max(x_ind)
    x_center = int((min_x_ind+max_x_ind)/2)

    min_y_ind = np.min(y_ind)
    max_y_ind = np.max(y_ind)
    y_center = int((min_y_ind+max_y_ind)/2)
    
    _,_,no_of_slices,no_of_frames = imgvol.shape
    
    start_slice_no = 1
    end_slice_no = no_of_slices-1
    
    start_frame_no = 1
    end_frame_no = no_of_frames-1
    
    final_vol = imgvol[:,:,start_slice_no:end_slice_no,start_frame_no:end_frame_no]
    
    save_path = out_path+patient_name+".h5"
    with h5py.File(save_path,'w') as f:
        f.create_dataset('inpVol',data = final_vol)
        f.create_dataset("x_center",data = [x_center])
        f.create_dataset("y_center",data = [y_center])

#     for slice_no in range(start_slice_no,end_slice_no):
        
#         frame_vol = imgvol[:,:,slice_no,start_frame_no:end_frame_no]
        
#         save_path = out_path+patient_name+"_"+"{}.h5".format(slice_no)
    
#         with h5py.File(save_path,'w') as f:
#             f.create_dataset('inpVol',data=frame_vol)


100%|█████████████████████████████████████████| 100/100 [02:06<00:00,  1.26s/it]


In [4]:
class slicer():
    
    def __init__(self,np_vol,crop_size):
        
        if np_vol.dtype == "complex128":
            self.volume = np.abs(np_vol)
        else:
            self.volume = np_vol
        
        self.vol_dim = np_vol.shape
        
        self.height_center = int(self.vol_dim[0]/2)
        self.width_center = int(self.vol_dim[1]/2)
        
        self.half_crop_size = int(crop_size/2)

    def vol_4d_slice_view(self,vol,slice_view,slice_no_x,slice_no_y,slice_no_t,slice_no_z,fig_size_x,fig_size_y):
        plt.figure(figsize=(fig_size_x,fig_size_y))
        if slice_view == 'x':
            plt.imshow(vol[slice_no_x,:,:,slice_no_t],cmap='gray')
            plt.show()
        elif slice_view == 'y':
            plt.imshow(vol[:,slice_no_y,:,slice_no_t],cmap='gray')
            plt.show()
        
        elif slice_view == 'z':
            plt.imshow(vol[:,:,slice_no_z,slice_no_t],cmap='gray')
            plt.show()

        elif slice_view == 't':
            plt.imshow(vol[:,:,slice_no_z,slice_no_t],cmap='gray')
            plt.show()
        
    def vol_3d_slice_view(self,vol,slice_view,slice_no_x,slice_no_y,slice_no_z,fig_size_x,fig_size_y):
        plt.figure(figsize=(fig_size_x,fig_size_y))
        if slice_view == 'x':
            plt.imshow(vol[slice_no_x,:,:],cmap='gray')
            plt.show()
        elif slice_view == 'y':
            plt.imshow(vol[:,slice_no_y,:],cmap='gray')
            plt.show()

        elif slice_view == 'z':
            plt.imshow(vol[:,:,slice_no_z],cmap='gray')
            plt.show()
            
    def slicer_view(self):
        if len(self.volume.shape) == 4:
            slice_no_x_max = self.volume.shape[0]-1
            slice_no_y_max = self.volume.shape[1]-1
            slice_no_z_max = self.volume.shape[2]-1
            slice_no_t_max = self.volume.shape[3]-1
            return interact(self.vol_4d_slice_view,vol = fixed(self.volume),slice_view = ['z','t','x','y'],slice_no_x = (0,slice_no_x_max,1),slice_no_y = (0,slice_no_y_max,1),slice_no_z = (0,slice_no_z_max,1),slice_no_t = (0,slice_no_t_max,1),fig_size_x=(5,8),fig_size_y=(5,8))
    
        elif len(self.volume.shape) == 3:
            slice_no_x_max = self.volume.shape[0]-1
            slice_no_y_max = self.volume.shape[1]-1
            slice_no_z_max = self.volume.shape[2]-1
            return interact(self.vol_3d_slice_view,vol = fixed(self.volume),slice_view = ['z','x','y'],slice_no_x = (0,slice_no_x_max,1),slice_no_y = (0,slice_no_y_max,1),slice_no_z = (0,slice_no_z_max,1),fig_size_x=(5,8),fig_size_y=(5,8))

In [8]:
f = h5py.File("/home/arun/datasets/ACDC_training/h5/patient066_4d.h5",'r')
np_data = f["inpVol"]

slice_viewer = slicer(np_data,crop_size = 154)
slice_viewer.slicer_view()

interactive(children=(Dropdown(description='slice_view', options=('z', 't', 'x', 'y'), value='z'), IntSlider(v…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

In [12]:
f.close()

## M&M dataset

In [1]:
import os
# import sys
import h5py
import numpy as np
import nibabel as nib
from tqdm import tqdm
from fnmatch import fnmatch

from matplotlib import pyplot as plt
from ipywidgets import interact,fixed

In [2]:
root = "/home/arun/datasets/M_M/Training/Labeled/"
out_path = root+"h5/"
pattern = '*_sa.nii.gz'

volume_list = []

dirnames = list(os.walk(root))

for path, subdirs, files in os.walk(root):

    for name in files:
        
        if fnmatch(name, pattern):
            volume_path = os.path.join(path, name)
            volume_list.append(volume_path)

print ("No of volumes:{}".format(len(volume_list)))

No of volumes:150


In [3]:
for iter,vol_path in enumerate(tqdm(volume_list)):

    inp_vol_path = vol_path
    patient_name = inp_vol_path.split("/")[-1].split(".")[0]

    splits = vol_path.split(".")
    gt_vol_path = splits[0]+"_gt."+splits[1]+"."+splits[2]

    imgvol  = nib.load(inp_vol_path)
    imgvol = imgvol.get_fdata()
    
    gtvol = nib.load(gt_vol_path)
    gtvol = gtvol.get_fdata()
    
    gtvol[gtvol > 0] = 1
    x_ind,y_ind,z_ind,_ = np.where(gtvol)
    
    min_x_ind = np.min(x_ind)
    max_x_ind = np.max(x_ind)
    x_center = int((min_x_ind+max_x_ind)/2)

    min_y_ind = np.min(y_ind)
    max_y_ind = np.max(y_ind)
    y_center = int((min_y_ind+max_y_ind)/2)
    
    _,_,no_of_slices,no_of_frames = imgvol.shape
    
    start_slice_no = 1
    end_slice_no = no_of_slices-1
    
    start_frame_no = 1
    end_frame_no = no_of_frames-1
    
    final_vol = imgvol[:,:,start_slice_no:end_slice_no,start_frame_no:end_frame_no]
    
    save_path = out_path+patient_name+".h5"
    with h5py.File(save_path,'w') as f:
        f.create_dataset('inpVol',data = final_vol)
        f.create_dataset("x_center",data = [x_center])
        f.create_dataset("y_center",data = [y_center])

100%|█████████████████████████████████████████| 150/150 [04:14<00:00,  1.70s/it]


In [4]:
class slicer():
    
    def __init__(self,np_vol,crop_size):
        
        if np_vol.dtype == "complex128":
            self.volume = np.abs(np_vol)
        else:
            self.volume = np_vol
        
        self.vol_dim = np_vol.shape
        
        self.height_center = int(self.vol_dim[0]/2)
        self.width_center = int(self.vol_dim[1]/2)
        
        self.half_crop_size = int(crop_size/2)

    def vol_4d_slice_view(self,vol,slice_view,slice_no_x,slice_no_y,slice_no_t,slice_no_z,fig_size_x,fig_size_y):
        plt.figure(figsize=(fig_size_x,fig_size_y))
        if slice_view == 'x':
            plt.imshow(vol[slice_no_x,:,:,slice_no_t],cmap='gray')
            plt.show()
        elif slice_view == 'y':
            plt.imshow(vol[:,slice_no_y,:,slice_no_t],cmap='gray')
            plt.show()
        
        elif slice_view == 'z':
            plt.imshow(vol[:,:,slice_no_z,slice_no_t],cmap='gray')
            plt.show()

        elif slice_view == 't':
            plt.imshow(vol[:,:,slice_no_z,slice_no_t],cmap='gray')
            plt.show()
        
    def vol_3d_slice_view(self,vol,slice_view,slice_no_x,slice_no_y,slice_no_z,fig_size_x,fig_size_y):
        plt.figure(figsize=(fig_size_x,fig_size_y))
        if slice_view == 'x':
            plt.imshow(vol[slice_no_x,:,:],cmap='gray')
            plt.show()
        elif slice_view == 'y':
            plt.imshow(vol[:,slice_no_y,:],cmap='gray')
            plt.show()

        elif slice_view == 'z':
            plt.imshow(vol[:,:,slice_no_z],cmap='gray')
            plt.show()
            
    def slicer_view(self):
        if len(self.volume.shape) == 4:
            slice_no_x_max = self.volume.shape[0]-1
            slice_no_y_max = self.volume.shape[1]-1
            slice_no_z_max = self.volume.shape[2]-1
            slice_no_t_max = self.volume.shape[3]-1
            return interact(self.vol_4d_slice_view,vol = fixed(self.volume),slice_view = ['z','t','x','y'],slice_no_x = (0,slice_no_x_max,1),slice_no_y = (0,slice_no_y_max,1),slice_no_z = (0,slice_no_z_max,1),slice_no_t = (0,slice_no_t_max,1),fig_size_x=(5,8),fig_size_y=(5,8))
    
        elif len(self.volume.shape) == 3:
            slice_no_x_max = self.volume.shape[0]-1
            slice_no_y_max = self.volume.shape[1]-1
            slice_no_z_max = self.volume.shape[2]-1
            return interact(self.vol_3d_slice_view,vol = fixed(self.volume),slice_view = ['z','x','y'],slice_no_x = (0,slice_no_x_max,1),slice_no_y = (0,slice_no_y_max,1),slice_no_z = (0,slice_no_z_max,1),fig_size_x=(5,8),fig_size_y=(5,8))

In [5]:
f = h5py.File("/home/arun/datasets/M_M/Training/Labeled/h5/A1D0Q7_sa.h5",'r')
np_data = f["inpVol"]

slice_viewer = slicer(np_data,crop_size = 154)
slice_viewer.slicer_view()

interactive(children=(Dropdown(description='slice_view', options=('z', 't', 'x', 'y'), value='z'), IntSlider(v…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

In [6]:
f.close()